####Reading data from customer table 

In [0]:
customer_df = spark.read.format("delta").load("dbfs:/FileStore/capstone/customerr_tbl/")

####Reading data from branches table


In [0]:
branches_df = spark.read.format("delta").load("dbfs:/FileStore/capstone/branches_tbl/")

####Writing data into bronze tables

In [0]:

customer_df.write.mode('overwrite').saveAsTable("bronze.Customer_tbl")
branches_df.write.mode('overwrite').saveAsTable("bronze.Branches_tbl")

###Reading streaming transaction data 

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Delta Stream Processing") \
    .getOrCreate()

inputPath = "dbfs:/FileStore/capstone/txn_tbl/latest/"
checkpoint_location = "dbfs:/FileStore/capstone/txn_tbl/checkpoint_location/"

# Read stream from Delta format
streamingDF = spark.readStream.format("delta").option("header", "true").load(inputPath)

def foreach_batch_function(df, epoch_id):
    # Write to a Databricks Delta table
    df.write.format("delta").mode("append").saveAsTable("bronze.txn_tbl")
    print(f"Batch for epoch {epoch_id} processed.")

# Write stream using foreachBatch to continuously trigger
query = streamingDF.writeStream.foreachBatch(foreach_batch_function).trigger(processingTime='10 seconds').option("checkpointLocation", checkpoint_location).start()

query.awaitTermination()


Batch for epoch 0 processed.
Batch for epoch 1 processed.
Batch for epoch 2 processed.
Batch for epoch 3 processed.
Batch for epoch 4 processed.
Batch for epoch 5 processed.
